In [315]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append("../../..")
import jtr.projects.foil2.sampling as sampling
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [3]:
num_preds = 1
dim = 5

weights = tf.Variable(tf.random_normal([num_preds,dim])) # [num_preds, dim]
sess.run(tf.initialize_all_variables())

In [4]:
num_samples = 10
num_important_samples = 10
samples = tf.random_uniform([num_samples, dim], minval=0.0, maxval=1.0) #[num_samples, dim]
scores = tf.matmul(weights, samples, transpose_b=True) # [num_preds, num_samples]
losses = tf.maximum(scores + 1, 0.0)
loss = tf.reduce_mean(losses, 1)

is_pos = tf.expand_dims(tf.to_float(tf.greater(weights, 0.0)),1)
inverse_transform_u = tf.random_uniform([num_important_samples, dim],minval=0.0001, maxval=0.9999)
important_samples = is_pos * tf.sqrt(inverse_transform_u) + (1.0 - is_pos) * (1.0 - tf.sqrt(inverse_transform_u))
importance_q_per_dim = is_pos * 2.0 * important_samples + (1.0 - is_pos) * (2.0 - 2.0 * important_samples)
importance_q = tf.exp(tf.reduce_sum(tf.log(importance_q_per_dim), 2)) # [num_preds, num_samples]
# important samples [num_preds, num_samples, dim]
# weights [num_preds, dim]

important_scores = tf.reduce_sum(tf.expand_dims(weights,1) * important_samples,2) # tf.matmul(weights, important_samples, transpose_b=True)
important_losses = tf.maximum(important_scores + 1, 0.0)
important_losses_weighted = important_losses / importance_q
important_loss = tf.reduce_mean(important_losses_weighted, 1)

# important_samples = 

sess.run(tf.shape(importance_q_per_dim))
sess.run(tf.reduce_sum(tf.log(importance_q_per_dim[0:2,0:3,:]),2))

array([[ 2.58681798,  2.11608577,  1.96498656]], dtype=float32)

In [5]:
sess.run(loss)

array([ 0.], dtype=float32)

In [6]:
sess.run((important_loss,importance_q,important_losses,losses))

(array([ 0.00393503], dtype=float32),
 array([[  7.38735437,   2.99969864,  10.04711246,   0.66703683,
           9.36897469,   1.31429386,   0.12127773,   5.08102417,
           7.59595633,   2.38967776]], dtype=float32),
 array([[ 0.        ,  0.        ,  0.38925904,  0.        ,  0.0056867 ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ]], dtype=float32),
 array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.18965346,  0.        ,  0.        ,  0.        ,  0.        ]], dtype=float32))

In [7]:
sess.run((tf.reduce_max(important_losses_weighted),
          important_loss, 
          tf.reduce_min(inverse_transform_u),tf.reduce_max(inverse_transform_u),
          tf.reduce_min(importance_q),
          tf.reduce_min(important_samples),
          tf.reduce_max(important_samples),
          tf.reduce_min(importance_q_per_dim)))

(0.12127987,
 array([ 0.01266411], dtype=float32),
 0.001440717,
 0.98168427,
 0.33561617,
 0.0092000961,
 0.96204323,
 0.075913548)

In [8]:
sess.run(tf.sqrt(2.3841858e-07))

0.00048828125

In [9]:
sess.run(losses)

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.53004754,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ]], dtype=float32)

In [10]:
sess.run(important_losses)

array([[ 0.45374691,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.50018966,  0.        ,  0.        ]], dtype=float32)

In [11]:
sess.run(is_pos[:,0,:])

array([[ 1.,  0.,  0.,  0.,  0.]], dtype=float32)

In [12]:
w_ = 8.0
eps = 0.000001
f1 = lambda:tf.minimum(w_,-eps)
f2 = lambda:tf.maximum(w_,eps)
w = tf.case([(tf.less(w_, 0.0),f1)],default=f2)
z = (tf.exp(w) - 1.0) 
u = tf.random_uniform([],0.0,1.0)
x = tf.log(z * u + 1) / w
p = tf.exp(x * w) * w / z
sess.run((x,p,z,tf.sign(w)))

(0.78942645, 1.4846642, 2979.958, 1.0)

In [317]:
# generate exponential

def sample_exp(w, num_samples):
    shape = tf.concat(0,((num_samples,),tf.shape(w)))
    f1 = tf.minimum(w,-eps)
    f2 = tf.maximum(w,eps)
    is_neg = tf.to_float(tf.less(w, 0.0))
    w_ = is_neg * f1 + (1.0 - is_neg) * f2
    u = tf.random_uniform(shape,0.0,1.0)
    z = (tf.exp(w_) - 1.0) 
    x = tf.log(z * u + 1) / w_
#     p = tf.reduce_prod(tf.exp(x * w_) * w_ / z,0)
    # TODO: replace with robuster version 
#     p = tf.exp(tf.reduce_sum(tf.log(tf.exp(x * w_) * w_ / z),1))
#     p = tf.exp(tf.reduce_sum(x * w_ + tf.log(w_) - tf.log(z),1))
    p = tf.exp(tf.reduce_sum(x * w_ + tf.log(tf.abs(w_)) - tf.log(tf.abs(z)),1))
    return x,p

def sample_exp_multi(w, num_samples):
    # w: [num_rows, dim]
    # returns: [num_samples, num_rows, dim] batch of samples drawn from a truncated exponential over [0,1] using w
    # as parameter
    shape = tf.concat(0,((num_samples,),tf.shape(w))) # [num_samples, num_rows, dim]
    f1 = tf.minimum(w,-eps)
    f2 = tf.maximum(w,eps)
    is_neg = tf.to_float(tf.less(w, 0.0))
    w_ = is_neg * f1 + (1.0 - is_neg) * f2
    u = tf.random_uniform(shape,0.0,1.0) #[num_samples, num_rows, dim]
    z = (tf.exp(w_) - 1.0) # [num_rows, dim]
    x = tf.log(z * u + 1) / w_ 
#     p = tf.reduce_prod(tf.exp(x * w_) * w_ / z,0)
    # TODO: replace with robuster version 
#     p = tf.exp(tf.reduce_sum(tf.log(tf.exp(x * w_) * w_ / z),1))
#     p = tf.exp(tf.reduce_sum(x * w_ + tf.log(w_) - tf.log(z),1))
    p_components = x * w_ + tf.log(tf.abs(w_)) - tf.log(tf.abs(z))
    p = tf.exp(tf.reduce_sum(p_components,2))
    return x,p

def sample_uniform(w, num_samples):
    shape = tf.concat(0,((num_samples,),tf.shape(w)))    
    x = tf.random_uniform(shape,0.0,1.0)
    return x, tf.constant(1.0)   

w = tf.Variable(tf.random_normal([3,10],stddev=0.5)) #[-5.,5.]
num_samples_1 = tf.placeholder(shape=[],dtype=tf.int32)# 100
num_samples_2 = tf.placeholder(shape=[],dtype=tf.int32)
x1,p1 = sampling.random_truncated_exponential(w, num_samples_1)
x2,p2 = sample_uniform(w, num_samples_2)

In [319]:
sess.run(tf.initialize_all_variables())

# sess.run((x1,p1))
# sess.run(tf.concat(0, (tf.shape(w),(2,))))
# sess.run((x1,p1))

In [318]:
def sum_by_importance_sampling(w, x, p):
    # w = [num_rows, dim]
    # x = [num_samples, num_rows, dim]
    scores = tf.reduce_sum(w * x, 2) # [num_samples, num_rows]
    losses = tf.maximum(scores + 1.0, 0.0)
    weighted = losses / p
    mean = tf.reduce_mean(weighted, 0) # [num_rows] 
    return mean
    
sum_1 = sum_by_importance_sampling(w,x1,p1)
sum_2 = sum_by_importance_sampling(w,x2,p2)

In [433]:
# sess.run((sum_1,x1,p1,w,w*x1,tf.reduce_mean((tf.reduce_sum(w*x1,1) + 1.0)/p1)),{num_samples_1:1, num_samples_2:1})
sess.run((sum_1,sum_2),{num_samples_1:4, num_samples_2:10000})

(array([ 1.04461384,  0.51943439,  0.66736847], dtype=float32),
 array([ 1.08322632,  0.46069068,  0.59924716], dtype=float32))

In [17]:
sess.run(sample_exp(5.0, []))

ValueError: Shapes must be equal rank, but are 1 and 0